In [12]:
# Import the dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import json
import gmaps
import os

# Setting the api key
from api_keys import g_key

In [13]:
# Create a new dataframe called vacation
file = "../Weatherpy/output/weatherpy.csv"
vacation = pd.read_csv(file)
vacation.head()

,City,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,pampas,-74.90,-12.40,49.48,81.0,33.0,2.77,PE,1.603172e+09
1,vila velha,-40.29,-20.33,69.80,94.0,75.0,2.24,BR,1.603171e+09
2,bluff,168.33,-46.60,51.01,71.0,37.0,13.00,NZ,1.603172e+09
3,karratha,116.85,-20.74,84.47,40.0,0.0,16.93,AU,1.603172e+09
4,ushuaia,-68.30,-54.80,35.60,93.0,75.0,26.40,AR,1.603172e+09


In [14]:
# Configure the api key
gmaps.configure(api_key=g_key)

In [15]:
# Finding the location of each city and setting the humidity value.
locations = vacation[["Latitude", "Longitude"]].astype(float)
humidity = vacation['Humidity']

In [16]:
# Creating a figure for gmaps
fig = gmaps.figure()

# Create a heat layer for the gmap with the weight being humidity.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=vacation['Humidity'].max(),
                                 point_radius = 2)

# Adds the heat layer.
fig.add_layer(heat_layer)

# Shows the figure.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Adds a new column in vacation dataframe.
vacation["Hotel Name"]= ""

# To find the comfortable vacation conditions 
hotel_df = vacation.loc[(vacation['Temperature'] > 70)&
                        (vacation['Temperature'] < 85)&
                        (vacation['Cloudiness'] == 0)&
                        (vacation['Wind Speed'] < 5)]
hotel_df

,City,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,doka,35.77,13.52,76.05,66.0,0.0,3.91,SD,1.603172e+09,
44,beloha,45.05,-25.17,80.20,31.0,0.0,4.59,MG,1.603172e+09,
103,tahta,31.50,26.77,73.40,35.0,0.0,2.24,EG,1.603172e+09,
118,forbes,148.02,-33.38,81.00,33.0,0.0,1.99,AU,1.603172e+09,
136,victoria,114.16,22.29,80.01,63.0,0.0,1.99,HK,1.603171e+09,
174,fomboni,43.74,-12.28,78.69,72.0,0.0,4.36,KM,1.603172e+09,
221,khandbari,87.20,27.37,80.74,46.0,0.0,4.27,NP,1.603172e+09,
233,garmsar,52.34,35.22,73.69,17.0,0.0,2.75,IR,1.603172e+09,
303,dire dawa,41.87,9.59,76.96,32.0,0.0,4.27,ET,1.603172e+09,
399,odweyne,45.06,9.41,80.35,19.0,0.0,4.94,SO,1.603172e+09,


In [23]:
# Creating a new url to find hotels within 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000

# Creating parameters to find specifically hotels.
param = {
    "radius" : radius,
    "types" : "lodgings",
    "key" : g_key
}

# To go through all the rows and pull the coordinates.
for index, rows in hotel_df.iterrows():
    lat = rows['Latitude']
    lng = rows['Longitude']

# Adding a new parameter, location
    param["location"] = f'{lat},{lng}'

# Creating an api request
    hotels = requests.get(base_url,params = param)
    hotels = hotels.json()

# Will append the hotel name and if there does not exist a hotel, it will skip.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels['results'][1]['name']
    except(KeyError, IndexError):
        print('Hotel not found')
    

In [24]:
# If there are empty values(Hotels not in vacinity)
# removes the value
hotel_df = hotel_df.replace(r'^\s*$', np.NaN, regex=True)
hotel_df = hotel_df.dropna(how = 'any')
hotel_df

,City,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,doka,35.77,13.52,76.05,66.0,0.0,3.91,SD,1.603172e+09,Джамия Дока
103,tahta,31.50,26.77,73.40,35.0,0.0,2.24,EG,1.603172e+09,Post Office - Shatoura
118,forbes,148.02,-33.38,81.00,33.0,0.0,1.99,AU,1.603172e+09,Lake Forbes Motel
136,victoria,114.16,22.29,80.01,63.0,0.0,1.99,HK,1.603171e+09,Mini Hotel Central
174,fomboni,43.74,-12.28,78.69,72.0,0.0,4.36,KM,1.603172e+09,School Of Mdjimbia
221,khandbari,87.20,27.37,80.74,46.0,0.0,4.27,NP,1.603172e+09,Makalu Arun Social Treks
233,garmsar,52.34,35.22,73.69,17.0,0.0,2.75,IR,1.603172e+09,Akbari
303,dire dawa,41.87,9.59,76.96,32.0,0.0,4.27,ET,1.603172e+09,Samrat Hotel
399,odweyne,45.06,9.41,80.35,19.0,0.0,4.94,SO,1.603172e+09,Somtel Office Odweyn
410,lucapa,20.74,-8.42,70.65,85.0,0.0,2.17,AO,1.603172e+09,HAMBUGUARIA ULISSSES DOS SANTOS


In [20]:
# Creates a template for the hotel markers
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# For loop to run through the dataframe.
# Pulls information and uses the template above.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Location of each hotel
locations = hotel_df[["Latitude", "Longitude"]]

In [21]:
# New figure
fig = gmaps.figure()

# New layer called hotel
# Add it to the figure
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Create a new figure and combine both layers.
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))